## Load tree via uproot

In [ ]:
import uproot
import awkward as ak


In [ ]:
fileName = '/udrive/staff/dnoonan/Skims/TTGamma_SingleLept_2016_skim.root'
_ttg_file = uproot.open(fileName)

In [ ]:
_ttg_file.keys()

In [ ]:
_ttg_tree = _ttg_file['Events']

In [ ]:
_ttg_tree.keys()

In [ ]:
arrays = _ttg_tree.arrays(['Jet_pt','Jet_eta'])

In [ ]:
arrays['Jet_eta']

In [ ]:
arrays['Jet_pt']

In [ ]:
jetSelect = ((arrays['Jet_pt']>30) & 
             (abs(arrays['Jet_eta'])<2.4))

In [ ]:
goodJetPt = arrays['Jet_pt'][jetSelect]

In [ ]:
ak.count(arrays['Jet_pt'],axis=1)

In [ ]:
ak.count(goodJetPt,axis=1)

#### Make plot of pt and eta of all jets, and selected jets

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(goodJetPt)

## Load data with Coffea

Use NanoEvents within Coffea to read and interpret file, loading branches

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema


In [ ]:
### This is currently not working on the skims, and checking with Coffea experts
# from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
# NanoAODSchema.warn_missing_crossrefs=True
# events = NanoEventsFactory.from_root(fileName, schemaclass=NanoAODSchema).events()
